In [1]:
import sys
import json

params = {'general':{'period':'am'}}
default = {'training_folder': '../..', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../..', 'params': {'general': {'period': 'am'}}}


In [2]:
def set_index(df):
    if 'index' in df.columns:
        df = df.set_index('index')
    else:
        df.index.name='index'
    return df

In [3]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
import numpy as np
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel


In [4]:
period = argv['params']['general']['period']
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)
training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'

if on_lambda:
    scenario_folder = training_folder +r'/inputs/'
    output_folder = training_folder + '/outputs/'
    model_folder = training_folder + '/model/'
else:
    scenario_folder = training_folder + '/scenarios/' + period + '/inputs/'
    output_folder = training_folder + '/scenarios/' + period + '/outputs/'
    model_folder = training_folder + '/scenarios/' + period + '/model/'
print('input folder: ', input_folder)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folder)
print('model folder : ', model_folder)


pt_folder = scenario_folder + 'pt/'
road_folder = scenario_folder + 'road/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

On Lambda :  False
input folder:  ../../inputs/
output folder:  ../../scenarios/am/outputs/
scen folder :  ../../scenarios/am/inputs/
model folder :  ../../scenarios/am/model/


In [5]:
var, ancestry = excel.read_var(file=input_folder+'/parameters.xlsx', scenario=period, return_ancestry=True)


# inputs

In [6]:
sm = stepmodel.read_zippedpickles(model_folder + 'network')

road_nodes: 100%|███████████████████████████████| 25/25 [00:00<00:00, 37.68it/s]


volumes

In [7]:
filepath = excel.get_filepath(input_folder + '{s}/volumes/volumes.csv', ancestry=ancestry, log=False)
volumes = pd.read_csv(filepath)
volumes = set_index(volumes)

In [8]:
period_dict={'am':'AM','pm':'PM','ip':'IP','op':'OP'}
volumes = volumes[volumes['time_period'] == period_dict[period]]

In [9]:
volumes['origin'] = 'zone_' + volumes['origin'].astype(str)
volumes['destination'] = 'zone_' + volumes['destination'].astype(str)

# congested road time

In [10]:
# scenario road time

In [11]:
road_times = pd.read_csv(road_folder+'road_times.csv')
road_times = set_index(road_times)
road_times['old_index'] = road_times.index
road_times.index = 'rlink_' + road_times.index.astype(str)

In [12]:
#reference road time

In [13]:
filepath = excel.get_filepath(input_folder + '{s}/road/road_times.csv', ancestry=ancestry, log=False)
ref_road_times = pd.read_csv(filepath)
ref_road_times = ref_road_times.set_index('index')['time'].to_dict()

In [14]:
road_times['time_ref'] = road_times.set_index('old_index').index.map(ref_road_times.get)

In [15]:
# freeflow road time

In [16]:
road_times['time_ff'] = sm.road_links['time']

# apply traffic

In [17]:
time_dict = road_times.set_index('old_index')['time_ff'].to_dict()
sm.links['time_ff'] = sm.links['road_link_list'].apply(lambda ls: sum([*map(time_dict.get, ls)]))


time_dict = road_times.set_index('old_index')['time'].to_dict()
sm.links['time_congestion'] = sm.links['road_link_list'].apply(lambda ls: sum([*map(time_dict.get, ls)]))

time_dict = road_times.set_index('old_index')['time_ref'].to_dict()
sm.links['time_ref'] = sm.links['road_link_list'].apply(lambda ls: sum([*map(time_dict.get, ls)]))

In [18]:
# broken mode, pour avoir sans rail et sans bus.mes deux segment
ref = sm.copy()

In [19]:
sm.links['jam_factor'] = sm.links['time_ff'] / sm.links['time_congestion']
sm.links['jam_factor'] = sm.links['jam_factor'].apply(lambda x: max(x,0.1)).apply(lambda x: min(x,1.2))
sm.links['time'] = sm.links['time'] / sm.links['jam_factor'] 


In [20]:
ref.links['jam_factor'] = ref.links['time_ff'] / ref.links['time_ref']
ref.links['jam_factor'] = ref.links['jam_factor'].apply(lambda x: max(x,0.1)).apply(lambda x: min(x,1.2))
ref.links['time'] = ref.links['time'] / ref.links['jam_factor'] 

# pathfinder

In [21]:
volumes_pt = volumes[volumes['vehicle_class'].isin(['RPAX','BPAX'])]
od_set = set(zip(volumes_pt['origin'], volumes_pt['destination']))


In [22]:
sm.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=True,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=True,
    path_analysis=False,
    od_set=od_set,
    engine='b', # b is faster!
)


start publicpathfinder
build_mode_combinations
find_broken_mode_paths


breaking modes: set() : 100%|█████████████████████| 4/4 [00:20<00:00,  5.15s/it]


path_analysis


In [23]:
def format_los(sm,volumes_pt):
    sm.pt_los['broken_modes'] = sm.pt_los['broken_modes'].apply(str)
    time_per_modes = sm.pt_los.groupby(['origin','destination','broken_modes'])['gtime'].agg(sum).unstack().reset_index()

    los = volumes_pt[['origin','destination','vehicle_class']].copy()
    los = los.drop_duplicates()

    los = los.merge(time_per_modes,on = ['origin','destination'])
    #los = los.merge(best_paths[['origin','destination','gtime']],on = ['origin','destination'])
    
    los['time']= np.nan
    idx = los['vehicle_class']=='BPAX'
    los.loc[idx,'time'] = los.loc[idx,"{'rail'}"]

    idx = los['vehicle_class']=='RPAX'
    los.loc[idx,'time'] = los.loc[idx,"{'bus'}"]

    los['all_walk'] = False
    los.loc[los['time'].isnull(),'all_walk'] = True
    try:
        los['time'] = los['time'].fillna(los["{'bus', 'rail'}"])
    except:
        los['time'] = los['time'].fillna(los["{'rail', 'bus'}"])
            
    return los

In [24]:
los = format_los(sm,volumes_pt)
los[['origin','destination','vehicle_class','time']].to_csv(output_folder +'pt_skim.csv')

In [25]:
sm.analysis_pt_los(walk_on_road=True)
sm.analysis_pt_route_type(hierarchy=[ 'car','rail', 'subway', 'tram', 'bus', 'walk'])

path_analysis: 100%|█████████████████| 201682/201682 [00:10<00:00, 18712.43it/s]


# ref pathfinder

In [26]:
ref.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=True,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=True,
    path_analysis=False,
    od_set=od_set,
    engine='b', # b is faster!
)

start publicpathfinder
build_mode_combinations
find_broken_mode_paths


breaking modes: set() : 100%|█████████████████████| 4/4 [00:20<00:00,  5.22s/it]


path_analysis


In [27]:
ref_los = format_los(ref, volumes_pt)
ref_los[['origin','destination','vehicle_class','time']].to_csv(output_folder +'ref_pt_skim.csv')


In [28]:
#ref_los[~ref_los['all_walk']]

In [29]:
sm.to_zippedpickles(model_folder +'los', omitted_attributes=['publicpathfinder'])

pt_los: 100%|███████████████████████████████████| 27/27 [00:10<00:00,  2.56it/s]
